In [1]:
from multiprocessing import Process, Queue
import os, time, random

In [11]:
# 写数据进程执行的代码:
def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

# 读数据进程执行的代码:
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)

In [12]:
q = Queue()
pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
# 启动子进程pw，写入:
pw.start()
# 启动子进程pr，读取:
pr.start()
# 等待pw结束:
pw.join()
# pr进程里是死循环，无法等待其结束，只能强行终止:
pr.terminate()

# 多进程处理

In [1]:
from multiprocessing import Process, Queue
import sys
import json
import base64
import time
import pyaudio
import wave

IS_PY3 = sys.version_info.major == 3

if IS_PY3:
    from urllib.request import urlopen
    from urllib.request import Request
    from urllib.error import URLError
    from urllib.parse import urlencode
    timer = time.perf_counter
else:
    from urllib2 import urlopen
    from urllib2 import Request
    from urllib2 import URLError
    from urllib import urlencode
    if sys.platform == "win32":
        timer = time.clock
    else:
        # On most other platforms the best timer is time.time()
        timer = time.time

In [18]:
from matplotlib import pyplot as plt
from IPython import display

In [15]:
API_KEY = 'kVcnfD9iW2XVZSMaLMrtLYIz'
SECRET_KEY = 'O9o1O213UgG5LFn0bDGNtoRN3VWl2du6'

# 普通版
#DEV_PID = 1536
#ASR_URL = 'http://vop.baidu.com/server_api'
# 极速版
DEV_PID = 80001
ASR_URL = 'https://vop.baidu.com/pro_api'

token = '24.9d7d643527ee67f41ed64f11e5b58444.2592000.1573186085.282335-15803531'

CHUNK = 16000
RATE = 16000
RECORD_SECONDS = 30

params = {'dev_pid': DEV_PID,
          'format': 'pcm',
          'rate': RATE,
          'token': token,
          'cuid': 'hmi-test',
          'channel': 1,
          'speech': 'none',
          'len': 0
          }

BACK_NOISE = 1000
SPEECH_LEVEL = 20000
KWS = ['记住', '记下来', '重点', '重要', '要考', '要提问']

In [4]:
Asr_valid = False
Asr_trig  = False
frames_int_pool = []

In [5]:
def raw_to_int(raw_data, lens=0):
    speech_data_ints = []
    data_sum = 0
    if lens==0:
        lens = len(raw_data)
    for i in range(0, lens, 2):
        speech_data_int = int.from_bytes(raw_data[i:i+2], byteorder='little', signed=True)
        speech_data_ints.append(speech_data_int)
        if abs(speech_data_int) > BACK_NOISE:
            data_sum += abs(speech_data_int)
    return speech_data_ints, data_sum

In [47]:
def recording(speech_datas, valid_q):
    global Asr_valid, frames_int_pool
    frames = []
    frames_int = []
    frames_sum = []

    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    print("开始录音，请说话......")

    # 先录音4s
    for i in range(4):
        data = stream.read(CHUNK)
        frames.append(data)
        data_int, data_sum = raw_to_int(data)
        frames_int.append(data_int)
        frames_sum.append(data_sum)
    print('录音启动完成')

    # 1s步长，5s窗口
    for i in range(0, RECORD_SECONDS):
        #print('\ni =', i)
        data = stream.read(CHUNK)
        frames.append(data)
        data_int, data_sum = raw_to_int(data)
        frames_int.append(data_int)
        frames_sum.append(data_sum)
        # 窗口内强度＞阈值，则认为有语音
        #print(frames_sum)
        if sum(frames_sum) > SPEECH_LEVEL:
            Asr_valid = True
            valid_q.put(True)
            #print('speech')
        else:
            Asr_valid = False
            valid_q.put(False)
            #print('silence')
        # 把窗口内的语音放入队列
        speech_datas.put(b''.join(frames))
        frames.pop(0)
        frames_sum.pop(0)

    speech_datas.put('Done')
    Asr_valid = False
    #print('录音完成，请闭嘴！')
    # 获取原始数据，用于调试
    #frames_int_pool = [i for item in frames_int for i in item]
    
    stream.stop_stream()
    stream.close()
    p.terminate()

In [54]:
def speech_process(speech_datas, valid_q):
    global Asr_valid, Asr_trig
    j = 0
    speech_data = 0
    while True:
        speech_data = speech_datas.get(True)
        #display.clear_output(wait=True)
        if speech_data == 'Done':
            print('\nOVER......')
            Asr_trig = False
            break
        print('\nj =', j)
        j += 1
        if not valid_q.get():
            print('Silience')
            Asr_trig = False
            continue
        params['speech'] = str(base64.b64encode(speech_data), 'utf-8')
        params['len'] = len(speech_data)
        post_data = json.dumps(params, sort_keys=False)
        req = Request(ASR_URL, post_data.encode('utf-8'))
        req.add_header('Content-Type', 'application/json')

        try:
            begin = timer()
            f = urlopen(req)
            result_str = str(f.read(), 'utf-8')
            print ("Request time cost %f" % (timer() - begin))
        except URLError as err:
            print('asr http response http code : ' + str(err.code))
            result_str = str(err.read(), 'utf-8')
        result = json.loads(result_str)

        if result['err_no'] == 0:
            s = result['result'][0]
            print(s)
            for i,key in enumerate(KWS):
                if key in s:
                    Asr_trig = True
                    print('trig:', key)
                    break
                elif i==len(KWS)-1:
                    Asr_trig = False
        else:
            print(result)
            Asr_trig = False

In [49]:
# 查看波形，用于调试
def show():
    global frames_int_pool
    plt.plot(frames_int_pool)
    for i in range(0, RECORD_SECONDS+5):
        plt.axvline(x=i*16000, color='#FFD700', linewidth=1)
    plt.show()

In [50]:
# 多进程
def main_mp():
    speech_datas = Queue()
    valid_q = Queue()
    pw = Process(target=recording, args=(speech_datas,valid_q))
    pr = Process(target=speech_process, args=(speech_datas,valid_q))
    # 启动子进程pw，写入:
    pw.start()
    # 启动子进程pr，读取:
    pr.start()
    # 等待pw结束，pr会自动结束:
    pw.join()

In [ ]:
main_mp()

In [11]:
# 单进程
speech_datas = Queue()
valid_q = Queue()
recording(speech_datas, valid_q)
speech_process(speech_datas, valid_q)

开始录音，请说话......
录音启动完成
i = 0
[890698, 0, 0, 0, 0]
speech
i = 1
[0, 0, 0, 0, 0]
silence
i = 2
[0, 0, 0, 0, 624653]
speech
i = 3
[0, 0, 0, 624653, 553224]
speech
i = 4
[0, 0, 624653, 553224, 18346]
speech
录音完成，请闭嘴！
len = 9.0
i = 0
Request time cost 0.667997
{'err_msg': 'speech quality error.', 'err_no': 3301, 'sn': '730715383671572795236'}
i = 1
Silience
i = 2
Request time cost 0.641500
电子系统。
i = 3
Request time cost 0.641821
电子系统设计这门课。
i = 4
Request time cost 0.734279
电子系统设计这门课很重要。
trig: 重要
OVER......


In [44]:
s = '这段话很重'

In [45]:
for i,key in enumerate(KWS):
    print(i)
    if key in s:
        Asr_trig = True
        print('trig:', key)
        break
    elif i==len(KWS)-1:
        Asr_trig = False

0
1
2
3
4
5


In [46]:
Asr_trig

False